In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as sio

In [216]:
class EnsemblePursuit():
    def __init__(self):
        self.nr_of_ensembles=None
        self.current_v=np.zeros(1)
        self.current_u=np.zeros(1)
        self.U=np.zeros(1)
        self.V=np.zeros(1)
        
    def calculate_cost_delta(self):
        cost_delta=np.maximum(np.zeros(self.X.shape[1]),self.current_v.T@self.X)/np.linalg.norm(self.current_v)**2-self.lambd
        cost_delta=cost_delta.reshape(self.X.shape[1],1)
        return cost_delta
    
    def fit_one_assembly(self):
        #Have to figure out a good initialization
        choose_neuron_idx=np.random.randint(0,self.X.shape[1],1)
        self.current_u=np.zeros((self.X.shape[1],1))
        self.current_u[choose_neuron_idx,0]=1
        self.current_v=self.X[:,choose_neuron_idx]
        max_delta_cost=1000
        i=0
        while max_delta_cost>0:
            cost_delta=self.calculate_cost_delta()
            excl_ind=np.where(self.current_u.flatten()==1)[0]
            masked_cost_delta=np.ma.array(cost_delta,mask=self.current_u)
            max_delta_neuron=np.ma.argmax(masked_cost_delta,fill_value=-1000)
            max_delta_cost=cost_delta[max_delta_neuron]
            if max_delta_cost>0:
                self.current_v=(self.current_v+self.X[:,max_delta_neuron].reshape(X.shape[0],1))/2
                self.current_u[max_delta_neuron,0]=1
            i+=1
        print(i)
        self.U=np.hstack((self.U,self.current_u))
        self.V=np.vstack((self.V,self.current_v.T))
        print(self.V.shape)
        
    #def plot_heatmaps(self):
        
        
    def fit_transform(self,X,lambd,n_ensembles=None):
        self.X=X
        self.lambd=lambd
        self.U=np.zeros((self.X.shape[1],1))
        self.V=np.zeros((1,self.X.shape[0]))
        self.fit_one_assembly()
    

In [129]:
X=sio.loadmat('/home/maria/Documents/EnsemblePursuit/natimg2800_M160825_MP027_2016-12-14.mat')['stim']['resp'][0][0]
print(X.shape)

(5426, 11449)


In [220]:
np.random.seed(7)
ep=EnsemblePursuit()
ep.fit_transform(X,0.5)

26
(2, 5426)
